# Study Selection
Complete version, contains record screening and full-text assessment 

In [2]:
import pandas as pd
import os
import json
import logging


from utils.General.quicker import Quicker, QuickerData, QuickerStage
from utils.Evidence_Assessment.paper import Paper
from utils.Evidence_Assessment.outcome import Outcome
from utils.Evidence_Assessment.evidence import Evidence
from utils.logging import setup_logging

# setup_logging()

# Hyperparameters
YOUR_CONFIG_PATH = 'config/config.json' #  your config.json file
YOUR_DATASET_PATH = 'data/2021ACR RA' # your dataset path
YOUR_QUESTION_DECOMPOSITION_PATH =  'data/2021ACR RA/Question_Decomposition' # your question decomposition folder
YOUR_LITERATURE_SEARCH_PATH = 'data/2021ACR RA/Literature_Search' # your literature search folder
YOUR_STUDY_SELECTION_PATH = 'data/2021ACR RA/Study_Selection' # your study selection folder
YOUR_EVIDENCE_ASSESSMENT_PATH = 'data/2021ACR RA/Evidence_Assessment' # your evidence assessment folder
YOUR_PAPER_LIBRARY_PATH = 'data/2021ACR RA/Paper_Library' # your paper library folder

disease = 'Rheumatoid Arthritis (RA)' # Disease name or clinical topic of your clinical question. 
pico_idx = "ef0e4f95" # PICO index of the clinical question, saved in your PICO_Information.json file

In [3]:
quickerdata_ls_path = os.path.join(YOUR_DATASET_PATH, f'quicker_data(PICO_IDX{pico_idx})_ls.json')
with open(quickerdata_ls_path, 'r') as f:
    quickerdata_ls = json.load(f)

In [4]:
quicker_data = QuickerData(disease=disease,pico_idx=pico_idx)
quicker = Quicker(
    config_path=YOUR_CONFIG_PATH,
    question_deconstruction_database_path=YOUR_QUESTION_DECOMPOSITION_PATH,
    literature_search_database_path=YOUR_LITERATURE_SEARCH_PATH,
    study_selection_database_path=YOUR_STUDY_SELECTION_PATH,
    evidence_assessment_database_path=YOUR_EVIDENCE_ASSESSMENT_PATH,
    quicker_data=quicker_data,
    paper_library_base=YOUR_PAPER_LIBRARY_PATH,
)


data_dict = dict(

    clinical_question=quickerdata_ls['clinical_question'],
    population=quickerdata_ls['population'],
    intervention=quickerdata_ls['intervention'],
    comparison=quickerdata_ls['comparison'],
    study=['randomized clinical trial'],

    search_results=quickerdata_ls['search_results'],
    # search_config=search_config,
)


quicker._add_data_to_quickerdata_for_test(
    stage=QuickerStage.LITERATURE_SEARCH,
    default_value=data_dict,
)

quicker.set_inclusion_exclusion_criteria(inclusion_criteria='', exclusion_criteria='')

In [5]:
print(quicker)
# print(quicker.quicker_data)
print(quicker.quicker_data.check_stage_state())
# print(quicker.quicker_data.not_none_data)


Quicker: STUDY_SELECTION
StageState.NOT_STARTED


In [6]:
from utils.Study_Selection.record_screening import screen_records
import time
from utils.Study_Selection.base import get_clinical_question_with_pico

logging.info("Run study selection")

quicker.execute_current_stage()

In [7]:
record_screening_method = quicker.config['study_selection']['record_screening_method']
quicker.quicker_data.to_json(
    os.path.join(YOUR_STUDY_SELECTION_PATH,f'Results/screening_records/{record_screening_method}/{pico_idx}')
)